In [14]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="capstone")
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="hyperdrive-xgboost")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [15]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Run Hyperdrive experiment

In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, quniform
import os

ps = RandomParameterSampling( {
    'max_depth': quniform(2, 10,1), # the maximum depth of each tree
    'min_child_weight': quniform(0, 4,1), # this is a Real instead of integer # if leaves with small amount of observations are allowed?
    'gamma' : uniform(0,0.15), # these 3 for model complexity. gemma is a threshold for gain of the new split. 
    'subsample': uniform(0.6,1.0), # (0.5,1)
    'colsample_bytree' : uniform(0.5,1.0), # these 3 for making model more robust to noise
    'reg_lambda' : uniform(0.0,0.9), # regularization lambda, reduces similarity scores and therefore lowers gain. reduces sensitivity to individual observations
    # 'reg_alpha' : Real(0.3,0.8), # regularization lambda, reduces similarity scores and therefore lowers gain. reduces sensitivity to individual observations
    'colsample_bylevel': uniform(0.60,1.0),
    'learning_rate': uniform(0.001,0.4), # 0.3 is default
    'max_delta_step': uniform(0.01,10.0),
    'n_estimators': quniform(10,4000,1)}) # the number of threes XGBoost makes


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core import Environment, ScriptRunConfig

project_folder = '.'
myenv = Environment.from_conda_specification(name = 'project1', file_path = '.azureml/project1.yml')

src = ScriptRunConfig(source_directory=project_folder, 
                      script='train_hyperdrive.py', 
                      compute_target=cpu_cluster, 
                      environment=myenv)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="rmse",
                             primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                             max_total_runs=220,
                             max_concurrent_runs=4)

hyperdrive_run = exp.submit(config=hyperdrive_config)
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-xgboost,HD_091ce256-d06c-47bb-b5fb-99f68e45a6a3,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [36]:
# clean up compute cluster
cpu_cluster.delete()

## Register datasets

In [2]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
train_url = ["https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/housing_after_preprocessing.csv"]
train = TabularDatasetFactory.from_delimited_files(train_url)
test_url = ["https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"]
test = TabularDatasetFactory.from_delimited_files(test_url)

In [5]:
df = train.take(5).to_pandas_dataframe()
df.head()

,Column1,Total_sqr_footage,GarageFinish,OverallCond,AllPorchSF,Condition1,OverallQual_3,AllSF,Neighborhood,SimplOverallCond,...,ScreenPorch,GrLivArea_log,GarageGrade,Fireplaces_log,BsmtFullBath_log,ExterQual_2,SimplOverallQual_3,GarageCars_2,SimplKitchenQual,y
0,0,2416,RFn,5,61,Norm,343,2566,CollgCr,2,...,0,7.444833,9.0,0.000000,0.693147,16,27,4,2,12.247699
1,1,2240,RFn,8,298,Feedr,216,2524,Veenker,3,...,0,7.141245,9.0,0.693147,0.000000,9,8,4,1,12.109016
2,2,2272,RFn,5,42,Norm,343,2706,CollgCr,2,...,0,7.488294,9.0,0.693147,0.693147,16,27,4,2,12.317171
3,3,1933,Unf,5,307,Norm,343,2473,Crawfor,2,...,0,7.448916,9.0,0.693147,0.693147,9,27,9,2,11.849405
4,4,2853,RFn,5,276,Norm,512,3343,NoRidge,2,...,0,7.695758,9.0,0.693147,0.693147,16,27,9,2,12.429220


## Run AutoML experiment

In [12]:
train = train.register(workspace=ws,
                                 name='train',
                                 description='training data')

test = test.register(workspace=ws,
                                 name='test',
                                 description='test data for kaggle submission')

In [22]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# changed validation set into n_cross_validation for review

automl_config = AutoMLConfig(
    experiment_timeout_minutes=60,
    enable_onnx_compatible_models=True,
    task='regression',
    primary_metric='normalized_root_mean_squared_error',
    training_data= train,
    compute_target=cpu_cluster,
    label_column_name='y',
    n_cross_validations=5)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [23]:
# Submit your automl run
from azureml.core import Workspace, Experiment
experiment=Experiment(ws, "automl")
automl_run = experiment.submit(automl_config, show_output = False)

Running on remote.


In [24]:
automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_6447f742-e7cc-4620-be3f-086c14a7ef61,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [32]:
# clean up compute cluster
cpu_cluster.delete()

In [ ]:
# Ignore below code

# # I have to convert the dataframe to a dataset since I'm running jupyter in VSCode locally
# from azureml.core import Dataset
# def upload(ws, df, name):
#     local_path = 'data/'+name+'.csv'
#     df.to_csv(local_path)
#     datastore = ws.get_default_datastore()
#     # upload the local file from src_dir to the target_path in datastore
#     datastore.upload(src_dir='data', target_path='data')
#     # create a dataset referencing the cloud location
#     return Dataset.Tabular.from_delimited_files(path = [(datastore, (local_path))])

# train_ds = upload(ws, train, 'train')
# test_ds = upload(ws, train, 'test')